## Weaviate Query Agent with Together AI

This notebook will show you how to define the Weaviate Query Agent as a tool through Together AI.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
2. Together AI API key: You can grab an API key [here](https://api.together.xyz/).
3. Install the Weaviate Agents package with `pip install weaviate-agents`
4. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [16]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import openai
import json

In [12]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["TOGETHER_AI_API_KEY"] = ""

### Define Query Agent function

In [13]:
def send_query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    # connect to your Weaviate Cloud instance
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"), 
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={ # add the API key to the model provider from your Weaviate collection, for example `headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"}` 
        }
    )

    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] 
    )
    return query_agent.run(query).final_answer

In [15]:
# test function

send_query_agent_request("How do I run Weaviate with Docker?")

"To run Weaviate with Docker, you should follow these steps:\n\n1. **Install Docker and Docker Compose**: Ensure that both Docker and Docker Compose are installed on your machine. You can find the installation guides for different operating systems like Mac, Windows, and Ubuntu Linux in the Docker documentation ([Docker for Mac](https://docs.docker.com/desktop/install/mac-install/), [Docker for Windows](https://docs.docker.com/desktop/install/windows-install/), and [Docker for Ubuntu](https://docs.docker.com/engine/install/ubuntu/)).\n\n2. **Obtain the Docker Compose File**: You can obtain a pre-configured `docker-compose.yml` file using the Weaviate configuration tool available on their website. Alternatively, you can use one of the example files from the Weaviate documentation.\n\n3. **Run the Docker Compose File**: Navigate to the directory where your `docker-compose.yml` file is located. Ensure it is named correctly as `docker-compose.yml`. Start the Weaviate setup by running the f

### Connect to Together AI and define Query Agent tool

In [17]:
client = openai.OpenAI(
    base_url = "https://api.together.xyz/v1",
    api_key = os.getenv("TOGETHER_AI_API_KEY") 
)

tools = [
  {
    "type": "function",
    "function": {
      "name": "send_query_agent_request",
      "description": "Send a query to the database and get the response",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "Any question or query to search for in the database"
          }
        },
        "required": ["query"]
      }
    }
  }
]

### Function calling loop and query time

In [8]:
def run_assistant(message, chat_history=None):
    if chat_history is None:
        chat_history = []
    
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)
    
    # Initialize messages with system prompt and user query
    messages = [
        {"role": "system", "content": "You are a helpful assistant that can access external functions. The responses from these function calls will be appended to this dialogue. Please provide responses based on the information from these function calls."},
        {"role": "user", "content": message}
    ]
    
    # Add chat history if available
    if chat_history:
        messages = chat_history + [{"role": "user", "content": message}]
    
    # Step 2: Generate tool calls (if any)
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo", # Together AI supports other models as well
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    # Get the assistant's message
    assistant_message = response.choices[0].message
    messages.append(assistant_message.model_dump())
    
    # Process tool calls if any
    while assistant_message.tool_calls:
        tool_calls = assistant_message.tool_calls
        
        if assistant_message.content:
            print("Tool plan:")
            print(assistant_message.content, "\n")
        
        print("Tool calls:")
        for call in tool_calls:
            function_name = call.function.name
            function_args = json.loads(call.function.arguments)
            print(f"Tool name: {function_name} | Parameters: {function_args}")
        print("="*50)
        
        # Step 3: Execute tools and get results
        tool_results = []
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            try:
                if function_name == "send_query_agent_request":
                    function_response = send_query_agent_request(function_args.get("query"))
                    # Add the function response to messages
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": function_name,
                        "content": str(function_response)
                    })
            except Exception as e:
                print(f"Error executing tool {function_name}: {str(e)}")
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": f"Error: {str(e)}"
                })
        
        # Step 4: Get the next response from the model
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message.model_dump())
    
    # Print final response
    print("Final response:")
    print(assistant_message.content)
    print("="*50)
    
    return messages

# Example usage
chat_history = run_assistant("How do I run Weaviate with Docker?")

Question:
How do I run Weaviate with Docker?
Tool calls:
Tool name: send_query_agent_request | Parameters: {'query': 'How do I run Weaviate with Docker?'}
Final response:
 To run Weaviate with Docker, follow these steps:

1. Install Docker and Docker Compose on your machine.
2. Obtain the Docker Compose file using the Weaviate configuration tool.
3. Run Docker Compose to start the containers in the background.
4. Check Weaviate's readiness using a cURL command.

By following these steps, you can successfully set up Weaviate using Docker for local development or experimentation.
